# Cox-Ingersoll-Ross Process

## 1. Introduction

Compared to the Ornstein-Uhlenbeck process, the Cox-Ingersoll-Ross (CIR) process allows for a stochastic diffusion component. This might be an attempt to improve the fit. To allow for sufficient comparison, the remaining aspects of the modelling process remain unchanged. As such, we once again use a MLE for the CIR process, which is defined as 

$$ dS_t = \alpha(\kappa - S_t)dt + \sigma \sqrt{dS_t} dW_t.$$

## 2. Maximum Likelihood Estimation 

In [1]:
import numpy as np

In [2]:
from ing.models.cir import CIR
from ing.fit.transition_density import ExactDensity
from ing.fit.mle_estimator import MLE
from ing.sim.simulator import SimulationSDE
from ing.plots.plot import plot_series_plotly, plot_density
from ing.utils.data_utils import (
    calculate_moments,
    conditional_expectation,
    expected_time_to_hit_target,
)

from ing.utils.data_utils import read_excel_to_series, strip_data

In [3]:
FILE_PATH = "../data/spread.xlsx"
COLUMN = "Spread"

In [4]:
df = read_excel_to_series(file_path=FILE_PATH)
spread = strip_data(df=df, column=COLUMN)

In [5]:
param_bounds = [(0, 50), (0, 50), (0.01, 50)]
guess = np.array([1, 0.1, 0.4])

In [6]:
freq = 252 * 12 * 8
dt = 1.0 / freq
model = CIR()

In [7]:
exact_est = MLE(
    sample=spread, param_bounds=param_bounds, dt=dt, density=ExactDensity(model=model)
).estimate_params(guess)

Initial Params: [1.  0.1 0.4]
Initial Likelihood: -126118.8457647932
`xtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 204, CG iterations: 135, optimality: 2.30e-01, constraint violation: 0.00e+00, execution time:  1.7 s.
Final Params: [0.05443819 0.74718357 1.7302386 ]
Final Likelihood: 44633.004627953196


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_differentiable_functions.py:231: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


In [8]:
params = exact_est.params
alpha, kappa, sigma = params[0], params[1], params[2]
alpha, kappa, sigma

(0.054438194235757915, 0.7471835728892986, 1.7302386007539634)

Based on the result of the MLE, we assume the spread can be fitted as an CIR with parameters 
$$ dS_t = \alpha(\kappa - S_t)dt + \sigma \sqrt{S_t} dW_t $$
$$ dS_t = 23.22(21.82 - S_t)dt + 8.07 \sqrt{S_t} dW_t $$ 

We can simulate this process using the Euler-Maruyama scheme, the most elementary discretization scheme as follows 
$$ S_{t_{n + 1}} = S_{t_{n}} + \alpha (\kappa - S_{t_{n}}) \Delta t  + \sigma \sqrt{\Delta t} \cdot \sqrt{S_t} \cdot \Delta W_n  $$ 

## 3. Plots

In [9]:
seed = 123
S0 = spread[0]
T = (len(spread) * 5) / ((252 * 8 * 60))
T = int(np.ceil((T)))

In [10]:
simulator = SimulationSDE(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
sample = simulator.sim_path()
sample = sample.flatten()
sample

array([20.38      , 20.33749672, 20.3193283 , ..., 32.86133199,
       32.93936668, 32.90736501])

In [11]:
TITLE = "CIR Fit"
X_AXIS = "Time"
Y_AXIS = "Spread"
LABELS = ["Real Spread", "CIR"]

In [12]:
plot_series_plotly(
    spread,
    sample,
    title=TITLE,
    labels=LABELS,
    xaxis_title=X_AXIS,
    yaxis_title=Y_AXIS,
    truncate=True,
)

In [13]:
moments_spread = calculate_moments(data=spread)

{'Kurtosis': 0.39893613722928967,
 'Mean': 21.707308807655405,
 'Skewness': -0.5474354025181073,
 'Variance': 1.4148241583285703}


In [14]:
moments_sim = calculate_moments(data=sample)

{'Kurtosis': -0.5196684633474709,
 'Mean': 26.300899011156236,
 'Skewness': 0.526570368831798,
 'Variance': 25.93375649684989}


We note that once again, the fit is poor. Suprisingly, poorer than the previous fit. 

## 4. Summary Statistics

In the final section, we compute both the conditional expectation of the spread and the expected time to hit the long term average. Additionally, plots are also provided 

First, the conditional expectation of the spread to hit the long term average. 

In [15]:
target_level = np.mean(spread)
target_level

21.707308807655405

In [19]:
cond_exp = conditional_expectation(
    data=sample, starting_point=0, T=900, target_level=target_level
)

[20.38       20.33749672 20.3193283  20.31107752 20.43106095 20.45828859
 20.40961877 20.30628589 20.30447425 20.25933664 20.39402574 20.36345428
 20.50458326 20.51413781 20.50553774 20.45094505 20.40740455 20.46849704
 20.45086896 20.47120152 20.43925429 20.48302203 20.41932548 20.45510755
 20.42148309 20.45852263 20.47990225 20.43998071 20.50169247 20.53460814
 20.57271667 20.48763088 20.51967579 20.54961955 20.53455451 20.5031992
 20.52378673 20.56973538 20.47842426 20.48778388 20.39387241 20.46097187
 20.43786206 20.41079969 20.4584108  20.4255571  20.31832034 20.30251642
 20.32037456 20.32664828 20.26061128 20.28439896 20.36534694 20.36798341
 20.41748327 20.34864699 20.29567687 20.18883985 20.14108337 20.09074978
 20.17924508 20.17346832 20.20870545 20.27278068 20.26430591 20.18824496
 20.13737694 20.16051941 20.16428627 20.11348588 20.13378487 20.05419967
 20.0026359  19.9925117  20.02497822 19.96289804 19.95906092 19.89786547
 19.88978612 19.8888807  19.88044876 19.86253353 19.

In [20]:
cond_exp

0.058888888888888886

In [21]:
exp_time = expected_time_to_hit_target(
    data=sample, starting_point=0, target_level=target_level
)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [19]:
exp_time

nan